In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/library/LSK.py
/kaggle/input/newlsk/LSK.py
/kaggle/input/all-province/all_province_data.xlsx


In [2]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.8 MB/s eta 0:00:00


In [3]:
# %%writefile MLLA1D.py


In [4]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import SAGEConv
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold


# 种子固定
np.random.seed(42)
torch.manual_seed(42)  # 设置 CPU 随机种子
torch.cuda.manual_seed(42)  # 设置 GPU 随机种子
torch.cuda.manual_seed_all(42)  # 如果使用多 GPU，设置所有 GPU 的随机种子
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
# -------------------------
# 1. 环境及数据准备
# -------------------------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

file_path = '/kaggle/input/all-province/all_province_data.xlsx'  # 请根据实际路径修改
data = pd.read_excel(file_path)

# 将时间列转换为年份（假设 '时间' 列格式可提取数字）
data['Year'] = data['时间'].str.extract('(\d+)').astype(int)

# 独热编码省份
data = pd.get_dummies(data, columns=['Province'])

# 选择特征
features = [
    'Year', 'Ammonia Nitrogen Emissions', 'Average Temperature', 
    'Average Years of Education per Capita', 'Chemical Oxygen Demand Emissions', 
    'Electricity Consumption', 'Geographic-Mean PM2', 
    'Government Expenditure on Environmental Protection', 
    'NOx Emissions', 'Number of Healthcare Institutions', 
    'Number of Healthcare Personnel', 'Oil Emissions', 
    'Per Capita Disposable Income', 'Resident Population', 
    'SO2 Emissions', 'Total Nitrogen Emissions', 
    'Total Phosphorus Emissions'
] + list(data.columns[data.columns.str.startswith('Province_')])

X = data[features]
y = data['Total CO2 emissions']

X = X.fillna(X.mean())

# -------------------------
# 2. 划分数据集
# -------------------------
# 将 2020, 2021, 2022 年的数据作为最终测试集，不参与交叉验证
X_test_2020 = X[X['Year'] == 2020]
X_test_2021 = X[X['Year'] == 2021]
X_test_2022 = X[X['Year'] == 2022]

y_test_2020 = y[X['Year'] == 2020]
y_test_2021 = y[X['Year'] == 2021]
y_test_2022 = y[X['Year'] == 2022]

# 其他年份作为训练集进行交叉验证（例如：<=2019）
X_train_all = X[~X['Year'].isin([2020, 2021, 2022])]
y_train_all = y[~X['Year'].isin([2020, 2021, 2022])]

# -------------------------
# 3. 定义模型
# -------------------------
class LSTMGraphSAGEModel(nn.Module):
    def __init__(self, num_node_features, hidden_dim, lstm_hidden_dim, out_dim, num_lstm_layers=2):
        super(LSTMGraphSAGEModel, self).__init__()
        
        # GraphSAGE layer
        self.conv1 = SAGEConv(num_node_features, hidden_dim)
        self.conv2 = SAGEConv(hidden_dim, hidden_dim)
        
        # LSTM layer
        self.lstm = nn.LSTM(input_size=num_node_features, hidden_size=lstm_hidden_dim, 
                            num_layers=num_lstm_layers, batch_first=True)
        
        # Fully connected layer for output
        self.fc = nn.Linear(hidden_dim + lstm_hidden_dim, out_dim)

    def forward(self, graph_data, lstm_input):
        # GraphSAGE part
        x, edge_index = graph_data.x, graph_data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        
        # LSTM part
        lstm_out, (hn, cn) = self.lstm(lstm_input)
        lstm_out = hn[-1]  # Use the last hidden state of the LSTM
        
        # Combine both GraphSAGE and LSTM outputs
        # 将 x.mean(dim=0) 扩展到与 batch_size 相同的维度
        x_expanded = x.mean(dim=0).unsqueeze(0).expand(lstm_out.size(0), -1)  # 使得 x_expanded 大小为 [batch_size, hidden_dim]
        combined = torch.cat([x_expanded, lstm_out], dim=-1)
        out = self.fc(combined)
        return out

# -------------------------
# 4. 定义构图函数
# -------------------------
def build_graph(x_array, threshold_percent=90):
    """
    根据给定的特征数组构建图，使用欧氏距离与指定百分位阈值构图。
    """
    distances = euclidean_distances(x_array)
    threshold = np.percentile(distances, threshold_percent)
    edge_index = np.array(np.where(distances < threshold))
    # 去除自环
    edge_index = edge_index[:, edge_index[0] != edge_index[1]]
    edge_index = torch.tensor(edge_index, dtype=torch.long)
    x_tensor = torch.tensor(x_array, dtype=torch.float)
    return Data(x=x_tensor, edge_index=edge_index)

# -------------------------
# 5. 五折交叉验证+早停+网格搜索
# -------------------------
candidate_learning_rates = [0.001, 0.005, 0.01]
candidate_patience_values = [10, 20, 30]
candidate_hidden_dims = [32, 64, 128]
candidate_lstm_hidden_dims = [32, 64, 128]

num_folds = 5
kfold = KFold(n_splits=num_folds, shuffle=True, random_state=42)
grid_search_results = []

print("\n====== Begin Grid Search ======")
for lr in candidate_learning_rates:
    for patience in candidate_patience_values:
        for hidden_dim in candidate_hidden_dims:
            for lstm_hidden_dim in candidate_lstm_hidden_dims:
                fold_r2_scores = []
                fold_best_epochs = []
                print(f"\nTesting combination: learning_rate={lr}, patience={patience}, hidden_dim={hidden_dim}, lstm_hidden_dim={lstm_hidden_dim}")
                for fold, (train_idx, val_idx) in enumerate(kfold.split(X_train_all)):
                    # 划分本折训练和验证数据
                    X_train_fold = X_train_all.iloc[train_idx].copy()
                    y_train_fold = y_train_all.iloc[train_idx].copy()
                    X_val_fold = X_train_all.iloc[val_idx].copy()
                    y_val_fold = y_train_all.iloc[val_idx].copy()
                    
                    scaler_x = MinMaxScaler()
                    X_train_fold_scaled = scaler_x.fit_transform(X_train_fold)
                    X_val_fold_scaled = scaler_x.transform(X_val_fold)
                    scaler_y = MinMaxScaler()
                    y_train_fold_scaled = scaler_y.fit_transform(y_train_fold.values.reshape(-1, 1)).ravel()
                    
                    train_data_fold = build_graph(X_train_fold_scaled).to(device)
                    val_data_fold = build_graph(X_val_fold_scaled).to(device)
                    y_train_fold_scaled_t = torch.tensor(y_train_fold_scaled, dtype=torch.float).to(device)
                    y_val_fold_t = torch.tensor(y_val_fold.values, dtype=torch.float).to(device)
                    
                    # Prepare LSTM input (time series features)
                    lstm_input_train = torch.tensor(X_train_fold_scaled.reshape(-1, 1, X_train_fold_scaled.shape[1]), dtype=torch.float).to(device)
                    lstm_input_val = torch.tensor(X_val_fold_scaled.reshape(-1, 1, X_val_fold_scaled.shape[1]), dtype=torch.float).to(device)

                    
                    model = LSTMGraphSAGEModel(num_node_features=X_train_fold_scaled.shape[1],
                                               hidden_dim=hidden_dim,
                                               lstm_hidden_dim=lstm_hidden_dim,
                                               out_dim=1).to(device)
                    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
                    criterion = torch.nn.MSELoss()
                    
                    best_val_r2 = float('-inf')
                    best_epoch = 0
                    no_improve_counter = 0
                    max_epochs = 1000
                    # max_epochs = 10
                    
                    for epoch in range(max_epochs):
                        model.train()
                        optimizer.zero_grad()
                        out = model(train_data_fold, lstm_input_train).squeeze()
                        loss = criterion(out, y_train_fold_scaled_t)
                        loss.backward()
                        optimizer.step()
                        
                        # 验证
                        model.eval()
                        with torch.no_grad():
                            preds_val = model(val_data_fold, lstm_input_val).squeeze().cpu().numpy()
                        preds_val_unscaled = scaler_y.inverse_transform(preds_val.reshape(-1, 1)).ravel()
                        val_r2 = r2_score(y_val_fold_t.cpu().numpy(), preds_val_unscaled)
                        
                        if val_r2 > best_val_r2:
                            best_val_r2 = val_r2
                            best_epoch = epoch + 1
                            no_improve_counter = 0
                        else:
                            no_improve_counter += 1
                        if no_improve_counter >= patience:
                            print(f"Fold {fold+1}: Early stopping at epoch {epoch+1} (patience={patience})")
                            break
                    fold_r2_scores.append(best_val_r2)
                    fold_best_epochs.append(best_epoch)
                    print(f"Fold {fold+1}: Best Epoch = {best_epoch}, Val_R² = {best_val_r2:.4f}")
            
            avg_r2 = np.mean(fold_r2_scores)
            avg_best_epoch = np.mean(fold_best_epochs)
            print(f"Combination lr={lr}, patience={patience}, hidden_dim={hidden_dim}, lstm_hidden_dim={lstm_hidden_dim}: Avg Val_R² = {avg_r2:.4f}, Avg Best Epoch = {avg_best_epoch:.2f}")
            grid_search_results.append({
                'learning_rate': lr,
                'patience': patience,
                'hidden_dim': hidden_dim,
                'lstm_hidden_dim': lstm_hidden_dim,
                'avg_val_r2': avg_r2,
                'avg_best_epoch': avg_best_epoch
            })

# 选择最佳超参数组合
best_combo = max(grid_search_results, key=lambda x: x['avg_val_r2'])
best_learning_rate = best_combo['learning_rate']
best_patience = best_combo['patience']
best_hidden_dim = best_combo['hidden_dim']
best_lstm_hidden_dim = best_combo['lstm_hidden_dim']
final_num_epochs = int(best_combo['avg_best_epoch'])  # 使用平均 best epoch 作为最终训练轮次

print("\n====== Grid Search Completed ======")
print("Best hyperparameters from grid search:")
print(best_combo)
print(f"Final training epochs set to: {final_num_epochs}")

Using device: cuda

====== Begin Grid Search ======

Testing combination: learning_rate=0.001, patience=10, hidden_dim=32, lstm_hidden_dim=32
Fold 1: Early stopping at epoch 39 (patience=10)
Fold 1: Best Epoch = 29, Val_R² = 0.0407
Fold 2: Early stopping at epoch 36 (patience=10)
Fold 2: Best Epoch = 26, Val_R² = 0.0220
Fold 3: Early stopping at epoch 29 (patience=10)
Fold 3: Best Epoch = 19, Val_R² = 0.0139
Fold 4: Early stopping at epoch 30 (patience=10)
Fold 4: Best Epoch = 20, Val_R² = 0.0306
Fold 5: Early stopping at epoch 26 (patience=10)
Fold 5: Best Epoch = 16, Val_R² = 0.0263

Testing combination: learning_rate=0.001, patience=10, hidden_dim=32, lstm_hidden_dim=64
Fold 1: Early stopping at epoch 35 (patience=10)
Fold 1: Best Epoch = 25, Val_R² = 0.0911
Fold 2: Early stopping at epoch 259 (patience=10)
Fold 2: Best Epoch = 249, Val_R² = 0.9761
Fold 3: Early stopping at epoch 33 (patience=10)
Fold 3: Best Epoch = 23, Val_R² = 0.0905
Fold 4: Early stopping at epoch 31 (patience=1

In [5]:
import pandas as pd
import numpy as np
import torch
import time
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 种子固定
np.random.seed(42)
torch.manual_seed(42)  # 设置 CPU 随机种子
torch.cuda.manual_seed(42)  # 设置 GPU 随机种子
torch.cuda.manual_seed_all(42)  # 如果使用多 GPU，设置所有 GPU 的随机种子
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# -------------------------
# 7. 使用最佳超参数以及数据训练最终模型，并对 2020-2022 预测
# -------------------------
# 对所有训练数据重新标准化
scaler_x_final = MinMaxScaler()
X_train_all_scaled = scaler_x_final.fit_transform(X_train_all)
scaler_y_final = MinMaxScaler()
y_train_all_scaled = scaler_y_final.fit_transform(y_train_all.values.reshape(-1, 1)).ravel()
train_data_final = build_graph(X_train_all_scaled).to(device)
y_train_all_scaled_t = torch.tensor(y_train_all_scaled, dtype=torch.float).to(device)

# Prepare LSTM input (time series features)
lstm_input_train = torch.tensor(X_train_all_scaled.reshape(-1, 1, X_train_all_scaled.shape[1]), dtype=torch.float).to(device)

results = []

print("\n====== Begin Final Training with Best Hyperparameters ======")
for i in range(10):
    print(f"\nFinal Training run {i+1}/10")
    start_time = time.time()
    
    # 使用修改后的 LSTM + GraphSAGE 模型
    model_final = LSTMGraphSAGEModel(num_node_features=X_train_all_scaled.shape[1],
                                     hidden_dim=best_hidden_dim,
                                     lstm_hidden_dim=best_lstm_hidden_dim,
                                     out_dim=1).to(device)
    optimizer_final = torch.optim.Adam(model_final.parameters(), lr=best_learning_rate)
    criterion_final = torch.nn.MSELoss()
    
    for epoch in range(final_num_epochs):
        model_final.train()
        optimizer_final.zero_grad()
        out_final = model_final(train_data_final, lstm_input=lstm_input_train).squeeze()  # 使用 LSTM 输入
        loss_final = criterion_final(out_final, y_train_all_scaled_t)
        loss_final.backward()
        # Apply gradient clipping (Flooding)
        # torch.nn.utils.clip_grad_norm_(model_final.parameters(), max_norm=1.0)  # 限制梯度最大范数为1.0
        
        optimizer_final.step()
        
        if (epoch+1) % 50 == 0:
            print(f"[Final Train] Epoch {epoch+1}/{final_num_epochs}, Loss: {loss_final.item():.4f}")
    
    model_filename = f'final_model_run_{i+1}.pth'
    torch.save(model_final.state_dict(), model_filename)
    
    def predict_and_evaluate(model, X_test, y_test):
        X_test_scaled = scaler_x_final.transform(X_test)
        data_test = build_graph(X_test_scaled).to(device)
        lstm_input_test = torch.tensor(X_test_scaled.reshape(-1, 1, X_test_scaled.shape[1]), dtype=torch.float).to(device)  # LSTM 输入
        y_test_t = torch.tensor(y_test.values, dtype=torch.float).to(device)
        
        model.eval()
        with torch.no_grad():
            preds_test = model(data_test, lstm_input=lstm_input_test).squeeze().cpu().numpy()
        preds_test_unscaled = scaler_y_final.inverse_transform(preds_test.reshape(-1, 1)).ravel()
        
        rmse = np.sqrt(mean_squared_error(y_test_t.cpu().numpy(), preds_test_unscaled))
        mae = mean_absolute_error(y_test_t.cpu().numpy(), preds_test_unscaled)
        r2 = r2_score(y_test_t.cpu().numpy(), preds_test_unscaled)
        nmae = mae / np.mean(y_test_t.cpu().numpy()) if np.mean(y_test_t.cpu().numpy()) != 0 else np.nan
        nrmse = rmse / np.mean(y_test_t.cpu().numpy()) if np.mean(y_test_t.cpu().numpy()) != 0 else np.nan
        return rmse, mae, r2, nmae, nrmse
    
    # 预测并评估 2020, 2021, 2022
    rmse_2020, mae_2020, r2_2020, nmae_2020, nrmse_2020 = predict_and_evaluate(model_final, X_test_2020, y_test_2020)
    rmse_2021, mae_2021, r2_2021, nmae_2021, nrmse_2021 = predict_and_evaluate(model_final, X_test_2021, y_test_2021)
    rmse_2022, mae_2022, r2_2022, nmae_2022, nrmse_2022 = predict_and_evaluate(model_final, X_test_2022, y_test_2022)
    
    end_time = time.time()
    training_time = end_time - start_time
    
    print(f"Run {i+1}: Training Time = {training_time:.2f} s")
    print(f"2020 - RMSE: {rmse_2020:.4f}, MAE: {mae_2020:.4f}, R²: {r2_2020:.4f}, NMAE: {nmae_2020:.4f}, NRMSE: {nrmse_2020:.4f}")
    print(f"2021 - RMSE: {rmse_2021:.4f}, MAE: {mae_2021:.4f}, R²: {r2_2021:.4f}, NMAE: {nmae_2021:.4f}, NRMSE: {nrmse_2021:.4f}")
    print(f"2022 - RMSE: {rmse_2022:.4f}, MAE: {mae_2022:.4f}, R²: {r2_2022:.4f}, NMAE: {nmae_2022:.4f}, NRMSE: {nrmse_2022:.4f}")
    
    results.append({
        'Run': i+1,
        'Training_Time (s)': training_time,
        'RMSE_2020': rmse_2020, 'MAE_2020': mae_2020, 'R²_2020': r2_2020, 'NMAE_2020': nmae_2020, 'NRMSE_2020': nrmse_2020,
        'RMSE_2021': rmse_2021, 'MAE_2021': mae_2021, 'R²_2021': r2_2021, 'NMAE_2021': nmae_2021, 'NRMSE_2021': nrmse_2021,
        'RMSE_2022': rmse_2022, 'MAE_2022': mae_2022, 'R²_2022': r2_2022, 'NMAE_2022': nmae_2022, 'NRMSE_2022': nrmse_2022
    })

# 保存结果
results_df = pd.DataFrame(results)
results_df.to_csv('model_training_results_with_time.csv', index=False)
print("\nResults saved to 'model_training_results_with_time.csv'.")



====== Begin Final Training with Best Hyperparameters ======

Final Training run 1/10
[Final Train] Epoch 50/73, Loss: 0.0014
Run 1: Training Time = 0.29 s
2020 - RMSE: 2558.6787, MAE: 2196.3499, R²: 0.9710, NMAE: 0.0610, NRMSE: 0.0711
2021 - RMSE: 3024.0015, MAE: 2603.0918, R²: 0.9622, NMAE: 0.0709, NRMSE: 0.0823
2022 - RMSE: 2624.2537, MAE: 2212.5034, R²: 0.9689, NMAE: 0.0602, NRMSE: 0.0714

Final Training run 2/10
[Final Train] Epoch 50/73, Loss: 0.0020
Run 2: Training Time = 0.28 s
2020 - RMSE: 2731.5483, MAE: 2392.8748, R²: 0.9669, NMAE: 0.0665, NRMSE: 0.0759
2021 - RMSE: 3132.6265, MAE: 2712.1819, R²: 0.9594, NMAE: 0.0738, NRMSE: 0.0853
2022 - RMSE: 2643.1125, MAE: 2208.8911, R²: 0.9685, NMAE: 0.0601, NRMSE: 0.0719

Final Training run 3/10
[Final Train] Epoch 50/73, Loss: 0.0024
Run 3: Training Time = 0.28 s
2020 - RMSE: 2439.3252, MAE: 2144.3914, R²: 0.9736, NMAE: 0.0596, NRMSE: 0.0677
2021 - RMSE: 2858.5986, MAE: 2457.9446, R²: 0.9662, NMAE: 0.0669, NRMSE: 0.0778
2022 - RMSE: 